In [2]:
import pandas as pd
from random import random, randint
import json

In [3]:
with open('gadm36_SSD_2.json','r') as f:
    t = f.read()
topo = json.loads(t)

In [5]:
df = pd.read_csv('South_Sudan_States.csv')

In [12]:
geoms = []
for geom in topo['objects']['gadm36_SSD_2']['geometries']:
    geoms.append(geom['properties'])
    
geo = pd.DataFrame(geoms)[['NAME_1','NAME_2']].drop_duplicates()

geo['State'] = geo['NAME_1']
geo['County'] = geo['NAME_2']

geo = geo[['State','County']].drop_duplicates()

In [14]:
df = geo

In [15]:
count_counties = len(df.County.unique())
count_states = len(df.State.unique())
print('There are {0} total states and {1} counties in South Sudan'.format(count_states, count_counties))

There are 10 total states and 45 counties in South Sudan


In [27]:
counties = {}
for county in df.County.unique():
    counties[county] = {'rainfall': random(), 'population': random(), 'infrastructure': random()}

In [35]:
def gen_year(state, county, year):
    obj = {}
    obj['State'] = state
    obj['County'] = county
    obj['Year'] = year
    
    county_obj = counties[county]
    rain_adj = county_obj['rainfall']
    pop_adj = county_obj['population']
    inf_adj = county_obj['infrastructure']
    
    obj['Rainfall'] = round(rain_adj*70 + random()*10,2)
    obj['Population'] = int(round(pop_adj*500000 + random()*15000,0))
    obj['Infrastructure'] = int(round(inf_adj*500 + random()*100,0))
    
    return obj

In [36]:
years = range(2000, 2019)

generated = []

for kk, vv in df.iterrows():
    for i in years:
        obj = gen_year(vv.State, vv.County, i)
        generated.append(obj)

In [37]:
output = pd.DataFrame.from_records(generated)
output = output[['Year','State','County','Population','Rainfall','Infrastructure']]

In [43]:
output.to_csv('~/Desktop/South_Sudan_Regional_Report.csv', index=False)

# Checking

In [44]:
rain = pd.DataFrame(output.groupby('County').Rainfall.mean())
pop = pd.DataFrame(output.groupby('County').Population.mean())
pop = pop.reset_index()
pop = pop.set_index('County')

In [53]:
rain = rain.reset_index().sort_values('Rainfall',ascending=False)
comb = rain.join(pop, on='County')

In [59]:
pop.describe()

,Population
count,45.000000
mean,306824.942690
std,138268.736980
min,11653.368421
25%,184626.578947
50%,326722.052632
75%,429908.315789
max,505202.526316


In [54]:
rain.describe()

,index,Rainfall
count,45.000000,45.000000
mean,22.000000,42.405719
std,13.133926,20.033611
min,0.000000,6.586316
25%,11.000000,25.546316
50%,22.000000,43.703158
75%,33.000000,58.198947
max,44.000000,74.537895


In [56]:
pop.mean()

Population    306824.94269
dtype: float64

In [62]:
comb[(comb['Rainfall'] > 58) & (comb['Population'] > 306824)]

,index,County,Rainfall,Population
0,3,Al Mayom,74.537895,332068.526316
5,14,Faring,66.633158,429908.315789
7,5,Aliab,66.040000,475447.263158
8,8,Aweil,65.281053,320670.684211
9,43,Yambio,62.215263,446042.052632
10,1,Al Leiri,61.028421,345999.631579
11,28,Pibor,58.198947,318153.315789


In [63]:
comb[(comb['Rainfall'] > 58)]

,index,County,Rainfall,Population
0,3,Al Mayom,74.537895,332068.526316
1,38,Tonj,72.904737,118722.526316
2,39,Wanjuk,70.757368,147946.684211
3,25,Nahr Atiem,67.845263,206266.842105
4,40,Warab,67.703684,135769.526316
5,14,Faring,66.633158,429908.315789
6,44,Yerol,66.050000,150317.105263
7,5,Aliab,66.040000,475447.263158
8,8,Aweil,65.281053,320670.684211
9,43,Yambio,62.215263,446042.052632
